In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import GRU_Models
from compact.knowledge_distillation import KnowledgeDistillation

In [2]:
conf = Config()
load_models = GRU_Models()
teacher_model = load_models.gru_4
teacher_model.load()
student_model = load_models.light_gru_4
kd = KnowledgeDistillation(
    teacher=teacher_model,
    student=student_model,
    device=conf.device,
    distillation="feature",
    rnn=True
)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/gru_4.pth!


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([32, 513, 1]) torch.Size([32])


In [5]:
acc, train_loss, val_loss = kd.train_kd(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Accuracy: 93.68%, Train loss: 0.0168, Val loss: 0.0950
Epoch: 2/10, Accuracy: 95.42%, Train loss: 0.0048, Val loss: 0.0654
Epoch: 3/10, Accuracy: 95.69%, Train loss: 0.0039, Val loss: 0.0588
Epoch: 4/10, Accuracy: 96.02%, Train loss: 0.0035, Val loss: 0.0552
Epoch: 5/10, Accuracy: 96.39%, Train loss: 0.0033, Val loss: 0.0515
Epoch: 6/10, Accuracy: 96.50%, Train loss: 0.0031, Val loss: 0.0493
Epoch: 7/10, Accuracy: 96.63%, Train loss: 0.0030, Val loss: 0.0480
Epoch: 8/10, Accuracy: 96.70%, Train loss: 0.0029, Val loss: 0.0460
Epoch: 9/10, Accuracy: 96.74%, Train loss: 0.0029, Val loss: 0.0448
Epoch: 10/10, Accuracy: 96.84%, Train loss: 0.0028, Val loss: 0.0443


In [6]:
# test teacher accuracy
loss, accuracy = teacher_model.evaluate(test_loader)
print(f"Teacher accuracy: {100*accuracy:.2f}%")

Teacher accuracy: 96.65%


In [7]:
# test student accuracy
loss, accuracy = student_model.evaluate(test_loader)
print(f"Student accuracy: {100*accuracy:.2f}%")

Student accuracy: 96.76%


In [8]:
student_model.save()

Checkpoint saved at /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/light_gru_4.pth
